# The Cut in Prolog

## Vision

For this project, I pursued learning about “the cut” in Prolog programming. The cut is a special built-in predicate used for controlling Prolog’s automatic backtracking, and it can be employed in different ways for different intended effects. I picked this topic because I am interested in Prolog as a logic programming language, and the cut seemed like an interesting new topic that would add deeply to my understanding. After learning about the cut, I explored its use through an application to route planning. I built a system that when given a pair of cities and a list of trains that connect those cities could compute whether or not the cities could be connected by a train route and if so, which trains needed to be taken in what order. This is a seemingly simple domain, but the programming itself was actually quite challenging and the result is practical and useful.

## Background

Prolog is a logic programming language that operates around the level of first-order logic, which is to say that it recognizes atomics, variables, predicates, and quantifiers (class***). The name Prolog comes from the phrase “programming in logic”, which shows just how focused on logic Prolog is. My project focused largely on how Prolog finds solutions within this logical structure, but the language has many practical applications apart from such a pure logical usage. In terms of artificial intelligence, Prolog has uses in theorem proving, expert systems, and natural language processing, among others (Wikipedia). Prolog works well for these uses because it is a declarative programming language, which means that Prolog statements are either facts or rules, which the user invokes with queries. This format makes Prolog relatively easy to read and understand. Most of the Prolog that was used in my project application was covered in our course study of Prolog, and I used Prolog because I find logic very interesting. I wanted to know more about practical applications and the powerful tools that Prolog has to offer, so I chose “the cut” as an interesting field I had not yet explored.

Integral to my projects work is “the cut”, which I learned about from Chapter 10 of Learn Prolog Now! by Blackburn, Bos, and Striegnitz. The cut is the built-in predicate !/0 which is used to control Prolog’s automatic backtracking. Some reasons why one might want to control backtracking is when one only wants to find one solution to a problem, to increase efficiency, to exclude some solutions, and for negation. The cut is used differently in each of these cases, although all uses of the cut fall into two categories: those that actually change the function of a program and those that cause side effects. The use for negation is a special case and has its own separate symbol.

The functionality of the cut is relatively simple: it locks the system into unification choices that have been made on the same line but to the left of this symbol. It does this because the !/0 predicate always passes, but it doesn’t allow choices made before it on the same line to be backtracked. So, should a program hit the ! symbol, evaluate the next goal and fail, then attempt to backtrack, the program will fail when the ! symbol is hit in the backtracking process. The program is allowed to backtrack past the !, but it must go back far enough to change a previous decision and not any of the ones made on the same line as the !. This change is best understood looking at the search tree of the program that includes the cut. When the cut is reached, all unexplored branches that branch off from all decisions made still on this line are cut off as options. The backtracking cannot explore these as options, so in order to change previous decisions it must backtrack past all decisions made on that line. In this way the unifications made on that line of programming cannot be changed after the cut has been hit. This functionality works the same in both uses of the cut, but does different things based on how and when it is employed.

The first use of the cut is cuts that do not change the function of a program. These are referred to as green cuts (Blackburn, Bos, and Striegnitz) and are most often used to improve the efficiency of a program. This occurs, for example, when a function (or rule) has multiple definitions so that there are different sets of goals that allow the rule to be achieved, but only one set of goals will be satisfied at a time. In this case, one can add a green cut to the end of the first goal set so that if a solution is found by the goals on that line, the program will recognize that as the only solution and not search for others. Learn Prolog Now has an example of this in Section 10.2 in the function defined on two lines: max(X,Y,Y)  :-  X  =<  Y,!. and max(X,Y,X)  :-  X>Y. (Blackburn***). For this function, Y is the maximum of X and Y if X is smaller than Y, or that is not the case and X must be bigger, so by placing a cut here the programmer is telling Prolog that if this first line is the case, the second is not, so we shouldn’t even consider it. This is more efficient because then Prolog will not backtrack and pursue looking for solutions on the second rule definition of max/3, thus saving time and computing power. This can also be considered an addition that tells the code not to consider some solutions, since if one line is the correct case, the other must not be, and the placement of the cut at the first line communicates that in some ways. However, it is important to note that this placement of the cut does not change the function of the rule definition. Nothing about the computation changes, other than that backtracking on this line is not allowed. This is what makes it a green cut.

All cuts that are not green cuts are red cuts, a name which signifies that the cut changes the functioning of the rule in some way that is likely quite significant. In many cases these cuts are considered dangerous, because if not well documented it may not be clear why the cut is present, but if it were to be removed the code would function differently or perhaps not at all (Blackburn***). The purpose of this kind of cut is difficult to explain because it may be employed for any number of reasons based on when and how the programmer wants to prevent backtracking. One specific use could be for exceptions, especially when combined with another built in predicate, fail. We see this functionality in this example from Section 10.3 of Learn Prolog Now: enjoys(vincent,X)  :-  big_kahuna_burger(X),!,fail. 
enjoys(vincent,X)  :-  burger(X). (Blackburn**). This example translates to the logical statement “Vincent enjoys all burgers except big kahuna burgers,” and is worked out in the code by first checking if the potential unification for X is a big kahuna burger and if it is, failing, otherwise moving on and simply checking if it is a burger before declaring that Vincent enjoys it. Interestingly, Prolog has some built in functionality to make this “negation as failure” even easier, by providing the \+ symbol which causes the goal to fail if the following term passes evaluation. Employing this, the previous two line rule can be rewritten on one line as “enjoys(vincent,X)  :-  burger(X), \+  big_kahuna_burger(X).” (Blackburn**). In this way, the cut definitely changes the way the program works by insisting that the code fails and cannot backtrack if X is a big kahuna burger. This is a unique way of exception handling that can only be done through the use of the cut, but it is only one example of the many ways that red cuts can be employed.


## Implementation and Results

My code for this project implements a solution to the problem posed in Learn Prolog Now! Exercise 10.4 by Blackburn, Bos, and Striegnitz. This problem provides a knowledge base of trains that connect a pair of cities and my code defines a predicate route/3 that given a pair of cities provides the route to reach one city from the other by train. This problem uses the cut in a few ways which make the problem more efficient and disregard recursive solutions. The first place the cut is used is in the first goal-rule pairing, “route(X,Y):- directTrain(X,Y), !.” This is a green cut because it does not change the way this rule functions, but it does stop the program from looking for additional solutions or a recursive runaway solution if a direct train route exists between the two cities. It just makes sense to take that train, which makes it the best solution. 

An interesting step in this application was that initially I was having problems getting my final route to be calculated all within one list. Because of the recursive nature of the my predicate, I was at first only able to get the correct list of cities in the correct order in a series of nested lists, where the last two cities visited were in the innermost list and the first city was the first item in a two item list where the second item was a nested list. To correct this, I needed to learn how to append items to lists, which I learned to do by reading Chapter 6 of Learn Prolog Now! titled “More Lists”. This chapter even had a definition for an append function, which I was able to read about, understand, and add to my code in order to get a neat, single list of all the cities visited in their correct order. 

At this point in time, my code does not fully work because my predicate only finds the route between two cities if they are listed in a specific order. The current version, for some orderings of cities, will suggest that there is a possibility for infinite recursion and thus stop itself without calculating the route. I believe that this issue can be fixed by employing a second different use of the cut, but I haven’t yet been able to figure out the exact usage. 

## Implications

In my study of Prolog, logic programming, and the cut, I couldn’t help but think about the tendency our society has to treat everything like data. We are in an age where we would like all things to be data, so that we could let computer analyze our world and tell us things about it. I worry for one that we are too willing to let computers do our thinking for us, but I am perhaps more concerned that this tendency implies that all things can be treated subjectively. The fact is, many things ought not to be treated as data, written into a knowledge base, and passed to a computer to make decisions about. We think that our computerized systems are free from bias because they don’t know what the data means, but that is exactly the problem: our data does have meaning, some of it so much meaning that it ought not to be treated objectively. I think by digitizing everything our society devalues life’s complexity and disregards the beauty in subjectivity. I think good Christian programmers should think critically about what types of decisions should or should not be delegated to computers, like ones that concern justice and require fair treatment for people. Even when we can build systems that will analyze situations for us, we should reserve some decisions for human insight, allowing room for justice and grace.